In [1]:
import pandas as pd
import numpy as np
import operator
import datetime
import pyecharts
from eunjeon import Mecab 
#from konlpy.tag import Hannanum
from collections import Counter
from collections import defaultdict

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
videos = pd.read_csv('data/videos.csv')

In [130]:
from pyecharts import Bar
bar = Bar(width=900, height=300)
bar.add("bar", 
        list(videos['channel_title'].value_counts()[:20].index), 
        list(videos['channel_title'].value_counts()[:20].values), 
        is_label_show=True,
        label_text_size =12,
        xaxis_label_textsize=10,
        xaxis_interval= 0,
        xaxis_rotate =20,
        bar_category_gap =2,
        is_stack=False,
        is_convert=False
       )
bar

In [314]:
tagger = Mecab() 
vocabs = []
for title in videos['title']:
    nouns = tagger.nouns(title)
    for noun in nouns:
        vocabs.append(noun)
        
for desc in videos['description']:
    if type(desc) == str:
        nouns = tagger.nouns(desc)
        for noun in nouns:
            vocabs.append(noun)

In [322]:
result = Counter(vocabs)

In [323]:
result = result.most_common(200)

result_dic = dict()

for i in range(len(result)):
    key = result[i][0]
    value = result[i][1]
    result_dic[key] = value

In [330]:
stopwords = ['한국', '대한민국', '반응', '해외', '외국인', '리액션', '이유', '수', '방송', '영상', '모음', '처음', '거', '일',
             '외국', '말', '위', 'ㅣ', '구독', '세계', '자막', '네티즌', '안녕', '사람', '상황', '시청', '채널', '구독', '오늘', 
             '것', '이것', '년', '댓글', '친구', '중', '때', '우리', '탄', '만', 'ㄷ', '편', '뮤', '급', '전', '국', '나',
            '부탁', '여러분', '소식', '최근', '이번', '분', '번', '등', '씨', '설정', '시', '디', '이', '사진', '월', '글',
             '생각', '그' , '미', '내', '때문', '제작', '유', '트', '후', '유튜브', '소개', '내용', '게', '라이브', '대' ,'이게', '원',
            '왕', '속']

for word in stopwords:
    try:
        del result_dic[word]
    except KeyError:
        pass

In [331]:
from pyecharts import WordCloud
wordcloud = WordCloud(width=800, height=620)
wordcloud.add("", result_dic.keys(), result_dic.values(), word_size_range=[20, 100] , shape='pentagon',  word_gap=5, rotate_step=5) 
wordcloud

In [190]:
countries = pd.read_excel('data/countries.xlsx')
countries_gni = pd.read_excel('data/countries_gni.xlsx')

In [191]:
countries_gni['country'] = [text.lstrip() for text in countries_gni['country']]

In [192]:
countries = pd.merge(countries, countries_gni, how='left', on='country')

In [193]:
countries_name = list(countries['country'])

In [181]:
countries_people = [ country + '인' for country in countries_name ]

In [182]:
countries_np = defaultdict()
for country in countries_name:
    if country in result.keys():
        countries_np[country] = result[country]
    if country + '인' in result.keys():
        countries_np[country] += result[country]

In [183]:
countries_np= sorted(countries_np.items(), key=operator.itemgetter(1), reverse=True)

In [184]:
#videos['date'] = [ datetime.datetime.strptime(videos.loc[i, 'date'][:-13], '%Y-%m') for i in range(len(videos)) ]

In [185]:
date_count = pd.DataFrame(videos['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]

In [225]:
tagger = Mecab() 

for i in range(len(videos)):
    countries_list = []
    
    title_nouns = tagger.nouns(videos.loc[i, 'title'])
    for noun in title_nouns:
        if noun in countries_name:
            countries_list.append(noun)
        if noun in countries_people:
            countries_list.append(noun[:-1])
    
    if type(videos.loc[i, 'description']) == str :
        desc_nouns = tagger.nouns(videos.loc[i, 'description'])
        for noun in desc_nouns:
            if noun in countries_name:
                countries_list.append(noun)
            if noun in countries_people:
                countries_list.append(noun[:-1])
            
    countries_list = list(set(countries_list))
    videos.loc[i, 'country'] = " ".join(countries_list)

In [211]:
videos.to_csv('data/videos_re.csv', encoding='utf-8-sig')

In [269]:
videos[videos['country']!='']

,Unnamed: 0,title,video_id,channel_title,channel_id,description,date,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,country
0,0,한국 예능 야구 레전드를 보고 현웃터진 미국인 반응,AEl5RAzF0lQ,BODA 보다,UCoCvTlU0KpNYwnMIgs7MPrA,KBO #한국야구 #예능짤 #MLB 오늘은 미국에서 온 김태균씨와 한국에 레전드 야...,2020-07-01,62865.00,755.00,21.00,0.00,114.00,미국
2,2,&quot;한국 스타일 무섭네!&quot; 한국의 건물사진에 깜짝놀란 스페인 반응과...,It69uJHJIuA,구네스북,UClHJcrFMOxCEPjjg6WlTmPw,한국의 건물사진에 깜짝놀란 스페인 반응과 네티즌 반응을 다뤘습니다. * 구네스북 그...,2020-07-01,249430.00,5237.00,196.00,0.00,390.00,스페인
3,3,한국 LG OLED TV의인기비결은???해외반응 – 일본이것이 한국의 기술이다!!일...,1SIA3ukXyhc,데보이,UC9Uf0ZCFKKy65uCOf6oCrOQ,해외반응#한국해외반응#일본 한국 LG OLED TV의 인기비결은??? 해외반응 – ...,2020-07-01,232245.00,2439.00,119.00,0.00,148.00,일본
4,4,한국 마스크를 처음 써보고 놀라운 반응을 보여주는 이탈리아 남자! 그 이유는?,3LwSAs3eeQA,레만 Reman [로마사는 한국남자],UC0MhsY1Nc467re-66o6xwRg,이탈리아 남자 알레한테 한국 마스크를 선물해주었습니다. 알레 반응이 정말 대박이네요...,2020-07-01,43407.00,1455.00,20.00,0.00,65.00,이탈리아
5,5,한국 길거리에 대놓고 있는 &#39;이것&#39;에 깜짝 놀란 외국인 반응,j-wy1mVDKBM,에이 스토리 A STORY,UCudezj2i6R94QOPqGJhbZwg,미국에서 온 신아린님은 한국 길거리에서 '이것'을 보고 처음에 너무 놀랐다는데요?!...,2020-07-01,4117907.00,29140.00,713.00,0.00,1079.00,미국
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,4800,"대만방송, 두려운 나라 이야기하던 중에 &quot;갑분 한국특집&quot;",E5ISiCtdF8w,션데일리 리서치,UCysrr_s8OwoZioh8XMOJn_g,대만방송에서 나라별로 어떤 나라가 강성해지는 것이 두려운가 라는 질문으로 여러나라의...,2021-06-01,556920.00,6284.00,202.00,0.00,1001.00,러시아 중국 대만 미국
4801,4801,[해외반응] 인도언론 “블랙핑크 BTS 인도 본토까지 KPOP 강타하다!” 해외네티...,Rx-0qbPLRvs,정치클라쓰TV,UCdzsoRaD0JzASmWaEYEZX1Q,인도언론 “블랙핑크 BTS 인도 본토까지 KPOP 강타하다!” 1:54 초부터! 해...,2021-06-01,62518.00,1183.00,43.00,0.00,97.00,인도
4802,4802,모두가 미쳤다고 말린 미국중학교에 태권도 수업을 넣자 일어난 충격적인 일,Z46SCGmcMQw,원더WONDER,UCrMGh7NV4sZlZQTaK0kWMAQ,우리나라 및 세계에서 일어나고 있는 다양한 이야기를 공유하는 채널입니다. 영미권 소...,2021-06-01,723461.00,15535.00,238.00,0.00,615.00,미국
4803,4803,&quot;한국은 북한보다 못 사는 나라&quot; 일본인 분석 글에 충격 받은 외...,_1S44pmqfwI,존크TV,UCitaXlLp5Oy0aKmH2_y0xFw,구독하기 : https://bit.ly/39beO6A 영상을 시청해주셔서 감사합니다...,2021-06-01,36468.00,1631.00,55.00,0.00,212.00,북한 일본


In [257]:
top10 = videos[videos['country']!=''].sort_values(by='viewCount', ascending=False)[:20]

In [266]:
top10[['title', 'description', 'country', 'viewCount']].to_csv('top10_videos.csv', encoding='utf-8-sig')

In [332]:
countries_videocnt = []
countries_viewCount = []
countries_likeCount = []
countries_dislikeCount = []
countries_favoriteCount =[]
countries_commentCount = []


for country in countries_name:
    cnt = len(videos[videos['country'].str.contains(country)]['viewCount'])
    vc = videos[videos['country'].str.contains(country)]['viewCount'].mean()
    lc = videos[videos['country'].str.contains(country)]['likeCount'].mean()
    dc = videos[videos['country'].str.contains(country)]['dislikeCount'].mean()
    fc = videos[videos['country'].str.contains(country)]['favoriteCount'].mean()
    cc = videos[videos['country'].str.contains(country)]['commentCount'].mean()
    
    countries_videocnt.append(cnt)
    countries_viewCount.append(vc)
    countries_likeCount.append(lc)
    countries_dislikeCount.append(dc)
    countries_favoriteCount.append(fc)
    countries_commentCount.append(cc)

In [333]:
countires_stat = pd.DataFrame({'country':countries_name, 'video_num':countries_videocnt, 'viewCount':countries_viewCount,'likeCount':countries_likeCount, 'dislikeCount':countries_dislikeCount,
             'favoriteCount':countries_favoriteCount, 'commentCount': countries_commentCount})

In [334]:
countires_stat.to_csv('countries_stat.csv', encoding='utf-8-sig')

In [335]:
countires_stat = pd.read_csv('countries_stat.csv')

In [336]:
countires_stat = pd.merge(countires_stat, countries, on='country', how='left')

In [339]:
pd.options.display.float_format = '{:.2f}'.format

In [340]:
top20_countries = countires_stat[countires_stat['video_num']>=5].dropna().sort_values(by='commentCount', ascending=False)[:50]

In [341]:
top20_countries.columns

Index(['Unnamed: 0', 'country', 'video_num', 'viewCount', 'likeCount',
       'dislikeCount', 'favoriteCount', 'commentCount', 'gni($)'],
      dtype='object')

In [354]:
import plotly.express as px


#국가별 영상 개수 top20
top20_countries = top20_countries.sort_values(by='video_num')
fig = px.bar(top20_countries, x="video_num", y="country", orientation='h', text='video_num', width=400, height=800)
fig.update_layout(uniformtext_minsize=10)
fig.show()

In [338]:
countires_stat.sort_values(by='commentCount', ascending=False)[:50]

,Unnamed: 0,country,video_num,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,gni($)
120,120,벨기에,1,22498511.00,632707.00,18252.00,0.00,11940.00,46052.40
194,194,나이지리아,2,1350104.50,18197.00,803.00,0.00,1693.00,2156.86
125,125,핀란드,4,1073385.00,16672.50,457.25,0.00,1402.75,48471.73
138,138,리투아니아,1,853450.00,23357.00,457.00,0.00,1296.00,19030.54
110,110,영국,124,652078.45,13672.17,364.29,0.00,1130.41,41290.95
23,23,몽골,6,407045.67,9419.17,294.50,0.00,865.50,3811.06
1,1,대만,76,384516.49,5243.62,185.47,0.00,849.28,26514.00
87,87,아르헨티나,4,1040414.00,9437.50,598.00,0.00,808.75,9653.77
155,155,헝가리,2,500931.50,11366.00,299.00,0.00,796.00,16020.42
10,10,미얀마,25,321149.72,7729.76,253.96,0.00,748.24,1356.12


In [249]:
import plotly.express as px
countires_stat = countires_stat.sort_values(by='viewCount', ascending=False)[1:].dropna()
fig = px.scatter(countires_stat, x="gni($)", y="viewCount", color = "video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [355]:
fig = px.scatter(countires_stat, x="gni($)", y="video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [358]:
countires_stat_re = countires_stat[countires_stat['video_num']<100]
fig = px.scatter(countires_stat_re, x="gni($)", y="video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [250]:
countires_stat_re = countires_stat[countires_stat['video_num']>5].dropna()

fig = px.scatter(countires_stat_re, x="gni($)", y="viewCount",  color = "video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [251]:
fig = px.scatter(countires_stat, x="gni($)", y="likeCount", color = "video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [252]:
fig = px.scatter(countires_stat, x="gni($)", y="dislikeCount", color = "video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [253]:
fig = px.scatter(countires_stat, x="gni($)", y="commentCount", color = "video_num", text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [267]:
fig = px.scatter(countires_stat_re, x="video_num", y="viewCount",  text="country")
fig.update_traces(textposition="bottom right")
fig.show()

In [100]:
df_america = videos[videos['country'].str.contains('미국')]
date_count = pd.DataFrame(df_america['date'].value_counts()).reset_index()
date_count = date_count.sort_values(by='index')
dates = [dt.date() for dt in list(date_count['index'])]
bar = Bar()
bar.add("bar", dates, list(date_count['date']), is_smooth=True, is_label_show=True)
bar